# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
llm_model = "gpt-3.5-turbo"

In [4]:
! pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


## LLMChain
- LLM prompt와 모델을 연결해주는 역할을 맡음.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
# 랭체인 object 정의 
llm = ChatOpenAI(temperature=0.9, model=llm_model)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt) # llm 모델과 prompt를 연결해준다. 

/Users/jieun/Projects/Langchain-Basics/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/jieun/Projects/Langchain-Basics/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
# 실행 
product = "Queen Size Sheet Set"
chain.run(product) # 한 번에 prompt를 주고 모델에 전달할 수 있으.ㅁ 

/Users/jieun/Projects/Langchain-Basics/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Royal Comfort Bedding Co.'

## SimpleSequentialChain

- sequential chain: 여러 체인들을 연결해주는 역할을 한다. 
- Simple Sequential Chain: 하나의 흐름으로 체인이 연결될 때 (인풋, 아웃풋이 1개 일 때)  

실습) (1) 회사 이름 정해주고, (2) 회사에 대한 설명을 설명해주는 2가지 체인을 연결하기

In [19]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [20]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [21]:
# sequential chain 만들기 
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

# 실행
overall_simple_chain.run(product)  # product = "Queen Size Sheet Set"



> Entering new SimpleSequentialChain chain...
"Royal Comfort Linens"
Royal Comfort Linens offers luxurious bedding and home decor products, designed to provide the ultimate in comfort and style.

> Finished chain.


'Royal Comfort Linens offers luxurious bedding and home decor products, designed to provide the ultimate in comfort and style.'

## SequentialChain
- Input과 Output 이 여러 개 일 때 쓰는 Chain

In [22]:
from langchain.chains import SequentialChain

In [27]:
# df 데이터 정의 
products = ['얼음 스쿱', '전동 칫솔', '애플 맥북 프로 M3']
reviews = ['얼음 스쿱이 플라스틱이라서 단단한 얼음을 풀 때마다 미세 플라스틱이 생깁니다. 비추, 꼭 스탠리스로 된 거 사세요 ', 
           '잇몸 까지 깨끗하게 닦아줍니다. 굿~', '배송이 엉망입니다. 200만원 넘는 제품인데 포장이 제대로 안되어 있어요. 반품하겠습니다. ']
df = pd.DataFrame()
df["product"] = products
df["reviews"] = reviews
df.head()

,product,reviews
0,얼음 스쿱,얼음 스쿱이 플라스틱이라서 단단한 얼음을 풀 때마다 미세 플라스틱이 생깁니다. 비추...
1,전동 칫솔,잇몸 까지 깨끗하게 닦아줍니다. 굿~
2,애플 맥북 프로 M3,배송이 엉망입니다. 200만원 넘는 제품인데 포장이 제대로 안되어 있어요. 반품하겠...


In [29]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [30]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [31]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [32]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [36]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review","language", "summary","followup_message"],
    verbose=True
)

In [38]:
review = df.reviews[0]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '얼음 스쿱이 플라스틱이라서 단단한 얼음을 풀 때마다 미세 플라스틱이 생깁니다. 비추, 꼭 스탠리스로 된 거 사세요 ',
 'English_Review': '"The ice scoop is made of plastic, so every time you scoop hard ice, tiny pieces of plastic are created. Not recommended, buy one made of stainless steel."',
 'language': 'Korean',
 'summary': 'The plastic ice scoop creates tiny pieces of plastic when scooping hard ice, making it not recommended compared to a stainless steel scoop.',
 'followup_message': '감사합니다. 플라스틱 아이스 스쿱의 단점을 알려주셔서 감사합니다. 스테인리스 스틸 스쿱과 비교해 볼 때 작은 플라스틱 조각이 발생한다는 것은 안전상 문제가 될 수 있습니다. 따라서 스테인리스 스틸 스쿱을 사용하는 것이 더 안전하고 적합하다는 점을 감안하여 앞으로는 그것을 사용하도록 하겠습니다. 다시 한번 감사드립니다.'}

## Router Chain

 - 한 체인의 결과에 따라서, 연결되는 체인이 다를 때 Router Chain 을 사용한다. 
 - Sequential Chain 보다 더 복잡하다. 

In [39]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [40]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [41]:
from langchain.chains.router import MultiPromptChain # 다양한 형식의 프롬프트 체인을 연결해야할 때 사용한다. 
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [42]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [43]:
# destination chain 정의 
destination_chains = {} # destination chain: router chain에 의해서 call될 것들 
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [44]:
# router 에 의해서 분류되지 않을 때, 예를 들어서 물리학, 수학, 컴퓨터 공학, 역사와 관련된 질문이 아닐 떄 
# default chain 이 사용된다. 
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [56]:
destinations

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [45]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it will be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [51]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [46]:
# 라우터 템플릿 만들기 - destination 들을 지정해준다. 
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# prompt TEMPLATE
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [48]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [49]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [57]:
#chain.run("Why does every cell in our body contain DNA?") # error: biology로 라우팅 됨
chain.run("what's the most popular Kpop song?") #DEFAULT



> Entering new MultiPromptChain chain...
None: {'input': "what's the most popular Kpop song?"}
> Finished chain.


'It is difficult to determine the most popular Kpop song as popularity can vary depending on different factors such as region, time, and personal preference. Some of the most popular Kpop songs in recent years include "Gangnam Style" by Psy, "Ddu-Du Ddu-Du" by BLACKPINK, "Boy With Luv" by BTS, and "Kill This Love" by BLACKPINK.'

Reminder: Download your notebook to you local computer to save your work.